In [ ]:
#https://cloud.google.com/speech-to-text/docs/sync-recognize#speech-sync-recognize-python
# !pip3 install --upgrade google-cloud-speech
# !pip3 install argparse

In [2]:
import os
import re
import io
from google.oauth2 import service_account
from google.cloud import speech
import pandas as pd

client_file = 'stutteringspeech-010b30e534ca.json'
credentials = service_account.Credentials.from_service_account_file(client_file)
client = speech.SpeechClient(credentials=credentials)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='cmn-Hans-CN'
)

def transcribe_GoogleCloud(filename):
    with io.open(filename, 'rb') as f:
        audio = speech.RecognitionAudio(content=f.read())
    response = client.recognize(config=config, audio=audio)
    try:
        return response.results[0].alternatives[0].transcript
    except:
        return ''

df_googlecloud = pd.DataFrame({
    'Filename': [],
    'Start_time': [],
    'End_time': [],
    'GoogleCloud': []
})

In [56]:
for i in range(1, 71):
    number = str(i).zfill(2)
    input_files = [f for f in os.listdir(f"segmented_audio/conversationA/00{number}")]
    for input_file in input_files:
        predicted = transcribe_GoogleCloud(f'segmented_audio/conversationA/00{number}/{input_file}')
        match = re.search(r'(\d+\.\d+)_(\d+\.\d+)\.wav$', input_file)
        start_time = match.group(1)
        end_time = match.group(2)
        df_googlecloud.loc[len(df_googlecloud)] = {'Filename': f'D00{number}_A','Start_time': start_time,'End_time': end_time,'GoogleCloud': predicted}

for i in range(1, 71):
    number = str(i).zfill(2)
    input_files = [f for f in os.listdir(f"segmented_audio/conversationB/00{number}")]
    for input_file in input_files:
        predicted = transcribe_GoogleCloud(f'segmented_audio/conversationB/00{number}/{input_file}')
        match = re.search(r'(\d+\.\d+)_(\d+\.\d+)\.wav$', input_file)
        start_time = match.group(1)
        end_time = match.group(2)
        df_googlecloud.loc[len(df_googlecloud)] = {'Filename': f'D00{number}_B','Start_time': start_time,'End_time': end_time,'GoogleCloud': predicted}

for i in range(1, 71):
    number = str(i).zfill(2)
    input_files = [f for f in os.listdir(f"segmented_audio/voice_command/00{number}")]
    for input_file in input_files:
        predicted = transcribe_GoogleCloud(f'segmented_audio/voice_command/00{number}/{input_file}')
        match = re.search(r'(\d+\.\d+)_(\d+\.\d+)\.wav$', input_file)
        start_time = match.group(1)
        end_time = match.group(2)
        df_googlecloud.loc[len(df_googlecloud)] = {'Filename': f'P00{number}','Start_time': start_time,'End_time': end_time,'GoogleCloud': predicted}

df_googlecloud.to_csv('predicted_transcription/GoogleCloud.csv')